Imports

In [ ]:
!pip install sktime
!pip install pandas
!pip install seaborn
!pip install sklearn


In [37]:
from sktime.datasets import load_from_arff_to_dataframe
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import zipfile
import shutil
import os
import random
from scipy.signal import find_peaks

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from scipy.stats import skew
from scipy.stats import kurtosis


creating datasets and spliting

In [ ]:
with zipfile.ZipFile("datasets.zip","r") as zip_ref:
    zip_ref.extractall()


In [38]:
X_train_sports, y_train_sports = load_from_arff_to_dataframe('RacketSports\RacketSports_TRAIN.arff')
X_test_sports, y_test_sports = load_from_arff_to_dataframe('RacketSports\RacketSports_TEST.arff')

data_mitbih_train = pd.read_csv('ECG\mitbih_train.csv', header=None)
data_mitbih_test = pd.read_csv('ECG\mitbih_test.csv', header=None)

X_train_mitbih = data_mitbih_train.iloc[:, 0:-1]
y_train_mitbih = data_mitbih_train.iloc[:, -1]

X_test_mitbih = data_mitbih_test.iloc[:, 0:-1]
y_test_mitbih = data_mitbih_test.iloc[:, -1]

data_ptbdb_abnormal = pd.read_csv('ECG\ptbdb_abnormal.csv', header=None)
data_ptbdb_normal = pd.read_csv('ECG\ptbdb_normal.csv', header=None)

data_ptbdb = pd.concat([data_ptbdb_abnormal, data_ptbdb_normal])

X_ptbdb = data_ptbdb.iloc[:, 0:-1]
y_ptbdb = data_ptbdb.iloc[:, -1]

X_train_ptbdb, X_test_ptbdb, y_train_ptbdb, y_test_ptbdb = train_test_split(X_ptbdb, y_ptbdb, test_size=0.2, random_state=80085)


In [ ]:
labels_sports = ['Badminton_Smash',
                'Badminton_Clear',
                'Squash_ForehandBoast',
                'Squash_BackhandBoast']

labels_mitbih = list(range(5))

labels_ptbdb = [0, 1]

datasets = [y_train_sports, y_test_sports, y_train_mitbih, y_test_mitbih, y_train_ptbdb, y_test_ptbdb]

plot_labels = [labels_sports, labels_mitbih, labels_ptbdb]
plot_titles = ['RacketSports', 'ECG Heartbeat Categorization Dataset - MIT-BIH', 'ECG Heartbeat Categorization Dataset - PTB Diagnostic']
plot_subtitles = ['Train Set', 'Test Set']

sns.set(rc={'figure.autolayout': True})

bg_color = '#E5E6F0'
color_line = '#0F101A'
colors = ['#5465FF', '#788BFF', '#9BB1FF', '#BFD7FF', '#D1EAFF', '#E2FDFF']

sns.set_theme(style="whitegrid", palette=sns.color_palette(colors))


**3.1.1**

Grafice ale frecvenței de apariție a fiecărei etichete (clase) în setul de date de antrenare / test

In [ ]:
results_dir = '3.1/1/'

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

for index in range(len(datasets)):
    # bar plot for class imbalance and cute pie on the site
    # print(datasets[index].shape)
    # print(plot_titles[index // 2])
    # print(plot_subtitles[index % 2])
    # plt.figure(facecolor=bg_color)

    fig, axes = plt.subplots(1, 2, figsize=(15, 7), facecolor=bg_color)
    fig.suptitle(plot_subtitles[index % 2] + ' - ' + plot_titles[index // 2], fontsize=20)

    sns.countplot(ax=axes[0], x=datasets[index], order=np.unique(plot_labels[index // 2]), edgecolor=color_line)
    axes[0].set_facecolor(bg_color)

    for p in axes[0].patches:
        axes[0].bar_label(container=axes[0].containers[0])

    counts = [len(datasets[index][datasets[index] == label]) for label in plot_labels[index//2]]
    axes[1].pie(counts, labels=plot_labels[index // 2], autopct='%1.1f%%')

    plt.savefig(results_dir + plot_titles[index // 2] + '_' + plot_subtitles[index % 2] + '.png')
    plt.show()


**3.1.2**

1) Câte un exemplu de serie pentru fiecare tip de acțiune din RacketSports - val de acc accelerometru si de giroscop

In [ ]:
random.seed(80085)
results_dir = '3.1/2/1/'

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

for label in labels_sports:
    indexes = random.sample([i for i in range(y_train_sports.shape[0]) if y_train_sports[i] == label], 6)

    fig = plt.figure(figsize=plt.figaspect(0.33), facecolor=bg_color)
    fig.suptitle('RacketSports ' + label + ' Exemplu - Accelerometru', fontsize=20)
    for i in range(6):

        x_acc = X_train_sports.to_numpy()[indexes[i]][0].to_numpy()
        y_acc = X_train_sports.to_numpy()[indexes[i]][1].to_numpy()
        z_acc = X_train_sports.to_numpy()[indexes[i]][2].to_numpy()

        ax = fig.add_subplot(2, 3, i + 1, projection='3d', facecolor=bg_color)
        ax.plot(x_acc, y_acc, z_acc, color=colors[0], zorder=-1)
        ax.plot(x_acc, y_acc, z_acc, color=colors[4], marker='*', markeredgecolor=colors[0], markeredgewidth=1, markersize=10, linestyle=' ')
    plt.tight_layout()
    plt.savefig(results_dir + 'RacketSports_' + label + '_Accelerometru' + '.png')
    plt.show()

    fig = plt.figure(figsize=plt.figaspect(0.33), facecolor=bg_color)
    fig.suptitle('RacketSports ' + label + ' Exemplu - Giroscop', fontsize=20)
    for i in range(6):

        x_giro = X_train_sports.to_numpy()[indexes[i]][3].to_numpy()
        y_giro = X_train_sports.to_numpy()[indexes[i]][4].to_numpy()
        z_giro = X_train_sports.to_numpy()[indexes[i]][5].to_numpy()

        ax = fig.add_subplot(2, 3, i + 1, projection='3d', facecolor=bg_color)
        ax.plot(x_giro, y_giro, z_giro, color=colors[0], zorder=-1)
        ax.plot(x_giro, y_giro, z_giro, color=colors[4], marker='*', markeredgecolor=colors[0], markeredgewidth=1, markersize=10, linestyle=' ')
    plt.tight_layout()
    plt.savefig(results_dir + 'RacketSports_' + label + '_Giroscop' + '.png')
    plt.show()


2) Afișați câte un exemplu de serie pentru fiecare categorie de aritmie din seturile de date MIT-BIH / PTB.

In [ ]:
random.seed(80085)
results_dir = '3.1/2/2/'

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# 'ECG Heartbeat Categorization Dataset - MIT-BIH', 'ECG Heartbeat Categorization Dataset - PTB Diagnostic'
time_sample = list(range(187))

for label in labels_mitbih:
    indexes = random.sample([i for i in range(y_train_mitbih.shape[0]) if y_train_mitbih[i] == label], 6)

    fig = plt.figure(figsize=plt.figaspect(0.33), facecolor=bg_color)
    fig.suptitle('ECG Heartbeat Categorization Dataset - MIT-BIH ' + str(label) , fontsize=20)
    for i in range(6):
        series = X_train_mitbih.to_numpy()[indexes[i]]

        ax = fig.add_subplot(2, 3, i + 1, facecolor=bg_color)
        ax.plot(time_sample, series, color=colors[0], zorder=-1)
    plt.tight_layout()
    plt.savefig(results_dir + 'ECG_Heartbeat_Categorization_Dataset_MIT_BIH_' + str(label) + '.png')
    plt.show()

for label in labels_ptbdb:
    indexes = random.sample([i for i in range(y_train_ptbdb.shape[0]) if y_train_ptbdb.to_numpy()[i] == label], 6)

    fig = plt.figure(figsize=plt.figaspect(0.33), facecolor=bg_color)
    fig.suptitle('ECG Heartbeat Categorization Dataset - PTB Diagnostic ' + str(label) , fontsize=20)
    for i in range(6):
        series = X_train_ptbdb.to_numpy()[indexes[i]]

        ax = fig.add_subplot(2, 3, i + 1, facecolor=bg_color)
        ax.plot(time_sample, series, color=colors[0], zorder=-1)
    plt.tight_layout()
    plt.savefig(results_dir + 'ECG_Heartbeat_Categorization_Dataset_PTB_Diagnostic_' + str(label) + '.png')
    plt.show()


3) Pentru seturile de date cu aritmii afișați un grafic al mediei și deviației standard
per unitate de timp, pentru fiecare clasă de aritmie. Media și deviația standard se
calculează peste toate exemplele (atât din train, cât și din train set).

In [ ]:
results_dir = '3.1/2/3/'

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

X_mitbih = pd.concat([X_train_mitbih, X_test_mitbih]).to_numpy()
y_mitbih = pd.concat([y_train_mitbih, y_test_mitbih]).to_numpy()


for label in labels_mitbih:
    indexes = [i for i in range(y_mitbih.shape[0]) if y_mitbih[i] == label]

    std_mitbih = np.std(X_mitbih[indexes], axis=0)
    mean_mitbih = np.mean(X_mitbih[indexes], axis=0)

    fig = plt.figure(facecolor=bg_color)
    fig.suptitle('ECG Heartbeat Categorization Dataset - MIT-BIH ' + str(label) , fontsize=20)
    ax = fig.add_subplot(1, 1, 1, facecolor=bg_color)
    ax.set_title('Standard Deviation and Mean')
    ax.plot(time_sample, std_mitbih, color=colors[3], linewidth=2.5)
    ax.plot(time_sample, mean_mitbih, color=colors[2], linewidth=2, zorder=-1)

    plt.savefig(results_dir + 'ECG_Heartbeat_Categorization_Dataset_MIT_BIH_mean_std_' + str(label) + '.png')
    plt.show()


for label in labels_ptbdb:
    indexes = [i for i in range(y_ptbdb.shape[0]) if y_ptbdb.to_numpy()[i] == label]
    std_ptbdb = np.std(X_ptbdb.to_numpy()[indexes], axis=0)
    mean_ptbdb = np.mean(X_ptbdb.to_numpy()[indexes], axis=0)

    fig = plt.figure(facecolor=bg_color)
    fig.suptitle('ECG Heartbeat Categorization Dataset - PTB Diagnostic ' + str(label) , fontsize=20)
    ax = fig.add_subplot(1, 1, 1, facecolor=bg_color)
    ax.set_title('Standard Deviation and Mean')
    ax.plot(time_sample, std_ptbdb, color=colors[3], linewidth=2.5)
    ax.plot(time_sample, mean_ptbdb, color=colors[2], linewidth=2, zorder=-1)
    plt.savefig(results_dir + 'ECG_Heartbeat_Categorization_Dataset_PTB_Diagnostic_mean_std_' + str(label) + '.png')

    plt.show()

4) Pentru setul de date RacketSports afișați distribuția valorilor per fiecare axă de
accelerometru și giroscop în parte / per acțiune (a se vedea un exemplu fictiv de
output în Figura 1). O astfel de analiză este utilă pentru a determina dacă există niște
șabloane imediate care se pot observa în termen de valori specifice pe x, y și z
pentru fiecare gest în parte. Aceste șabloane (e.g. intervale de valori specifice) pot fi
folosite în etapa de definire a atributelor și pot totodată informa dacă problema este
una ușoară (șabloane de valori clar diferențiabile per gest) sau grea (distribuții
similare per fiecare gest).


In [ ]:
data_sports = pd.concat([X_train_sports, X_test_sports])
data_sports['label'] = np.append(y_train_sports, y_test_sports)

columns_sports = data_sports.columns
print(columns_sports)

# for label in labels_sports:
#     indexes = [i for i in range(y_train_sports.shape[0]) if y_train_sports[i] == label]
#     data_sports_parsed = pd.DataFrame()
#     working_data = X_train_sports.iloc[indexes]
#     # print(working_data.columns)
#     for col in columns_sports:
#         values = np.zeros((30 * len(working_data)))
#         for i in range(30 * len(working_data)):
#             values[i] = working_data['dim_0'].to_numpy()[i//30][i % 30]
#         data_sports_parsed[col] = values

#     for col in columns_sports:
#         sns.histplot(data=data_sports_parsed, x=col, color=)
#     plt.show()



# print(values)
# sns.histplot(data=X_train_sports, kde=True)
# Unde <nume variabila target> reprezinta numele atributului din setul de date ce identifică
# eticheta gestului, <numar clase posibile> este 4 pentru setul de date RacketSport, iar <nume
# dimensiune x, y, sau z> este numele atributului pe care îl dați la încărcare setului de 300
# valori care corespund axei x, y sau z după caz.

# print(X_train_sports)
# print(y_train_sports.type)

data_sports_parsed = pd.DataFrame()
for col in columns_sports:
    values = np.zeros((30 * len(data_sports)))
    if col != 'label':
        for i in range(30 * len(data_sports)):
            values[i] = data_sports[col].to_numpy()[i//30][i % 30]
        data_sports_parsed[col] = values
    else:
        values = np.empty((30 * len(data_sports)), dtype='object')
        for i in range(30 * len(data_sports)):
            values[i] = data_sports[col].to_numpy()[i//30]
        print(values)

for col in columns_sports:
    if col != 'label':
        sns.histplot(data=data_sports_parsed, x=col)
plt.show()

# sns.displot(data_sports_parsed, x='dim_0', hue='label', binwidth=3, height=3, facet_kws=dict(margin_titles=True))
# g = sns.FacetGrid(data_sports_parsed, col='label', height=3.5, aspect=.65)
# g.map(sns.histplot, "dim_0")

plt.show()


3.2.1

Pentru analiza celor două seturi de date veți folosi următorii algoritmi:


● RandomForest - folosiți implementarea din scikit-learn [0.5p]
● GradientBoosted Trees - folosiți implementarea din biblioteca xgboost [0.75p]
● SVM - folosiți implementarea din scikit-learn [0.75p]



Folosiți înțelegerea datelor câștigată la pasul 3.1 pentru a determina dacă este necesară
standardizarea datelor. Acest pas este unul des întâlnit etapă de pre-procesare a datelor
înainte de antrenarea unui clasificator, în vederea uniformizării valorilor numerice aferente
fiecărui tip de atribut (e.g. nu este dorit ca unele atribute sa aibă valori de ordinul miilor, iar
altele de ordinul unităților).



Partea de extragere a atributelor propusă în secțiunea 3.2.2 poate duce la un număr mare
de atribute extrase. Frecvent se întâmplă ca nu toate atributele să aibă o contribuție
importantă în cadrul predicției.
Ca atare, investigați aplicarea tehnicilor de selectare a atributelor (eng. Feature selection)
oferite în scikit-learn. Folosiți cel puțin una din metodele Variance Threshold sau Select
Percentile.


Fiecare algoritm din cei propuși are o serie de hiper-parametrii care influențează
funcționarea acestuia. Pentru a găsi valorile potrivite pentru aceștia veți folosi o procedură
de căutare a hiper-parametrilor pe bază de Grid Search cu Cross Validation.
Setul minim de hiper-parametrii de căutat este:


● SVM: tipul de kernel, parametru C de regularizare
● RandomForest: numărul de arbori, adâncimea maximă a unui arbore, procentul din
input folosit la antrenarea fiecărui arbore
● GradientBoostedTrees: numărul de arbori, adâncimea maximă a unui arbore,
learning rate


Evaluarea algoritmilor
În raportul vostru trebuie sa prezentați următoarele:
● Rezultatul procedurii de feature selection: numărul total de feature-uri considerate și
numărul total de feature-uri utilizate la antrenare (ca urmare a procedurii de feature
selection)
● Pentru fiecare algoritm, realizați un tabel în care să prezentați media si varianța
pentru acuratețea generală de clasificare, precizie / recall / F1 la nivelul fiecărei
clase în parte
○ Pe linii va fi indexată configurația de hiper-parametrii rezultată din procedura
de GridSearch.
○ Pe coloane vor fi prezentate metricile cerute
○ Relevați prin bolduire valorile maxime pentru fiecare metrică
● Pentru cea mai bună variantă a hiper-parametrilor, pentru fiecare algoritm,
realizați o matrice de confuzie peste clase.

3.2.2


A. Extragere de atribute pentru RacketSports [1p]
Datele din RacketSports reprezinta valori continue pe fiecare axă ale unor acțiuni cu 300 de
observații per serie. Ca atare, setul de atribute agregate pe care le putem extrage se referă
la analiza statistică a semnalelor pe fiecare axă.
Sugestii de atribute statistice de extras per serie (sau sub-fereastră a seriei) / per axă:


● medie

● abaterea standard

● abaterea medie absolută

● valoare minimă

● valoare maximă

● diferenta de valori maxime si minime

● mediană

● abaterea mediană absolută

● intervalul intercuartil

● Număr de valori negative

● Număr de valori pozitive

● număr de valori peste medie

● număr de vârfuri

● Energia semnalului

○ Energia unui semnal pe fiecare axă este calculată luând media sumei

pătratelor valorilor dintr-o fereastră pe axa respectivă.

● Asimetrie (skewness)

● Curtoză (kurtosis)

● Accelerația medie rezultantă

○ Accelerația medie rezultantă peste fereastră este calculată luând media
rădăcinilor pătrate ale valorilor din fiecare dintre cele trei axe pătrate și
adunate împreună.

● Aria mărimii semnalului

○ Aria mărimii semnalului este definită ca suma valorilor absolute ale celor
trei axe mediate pe o fereastră.



În afară de atributele statistice, pentru secvențe numerice interpretate ca semnale se pot
calcula atribute extrase pe baza interpretării în regim de frecvență a semnalului (i.e.
aplicând o transformată Fourier).
Pentru exemple și cod de extragere a atributelor, atât statistice cât și Fourier, urmăriți acest
tutorial.
B. Extragere de atribute pentru Seturile de date cu Aritmii [1p]
Pentru seturile MIT-BIH si PBT veți încerca să antrenați algoritmii de la punctul 3.2.1
folosind:
● Direct datele de intrare (fiecare din cele 187 de puncte ale unei serii
reprezentând bătaia inimii este un atribut)
● Atributele statistice descrise la litera A.
NOTĂ: extragerile de atribute prezentate mai sus pot fi aplicate pe toată lungimea
seriei sau pot fi aplicate pe ferestre de lungime H, unde H < lungimea secvenței.
Aceasta înseamnă că puteți împărți secvența voastră în subsecvențe (cu sau făra
suprapunere) și să calculați atributele pe fiecare subsecvență în parte.

In [ ]:
class feature_extractor:
    def __init__(self, time_series, labels, W=20, sample_step=15):
        self.time_series = time_series
        self.labels = labels
        self.W = W

        # self.data = [(self.time_series[index][i:i + self.W], index)
        #              for index in range(len(self.time_series))
        #              for i in range(0, len(self.time_series[index]), sample_step)]

        # self.indexes = [self.labels[i // len(self.time_series)] + '_' + str(i % len(self.time_series)) for i in range(len(self.data))]

        self.data = []
        self.indexed_labels = []

        for i in range(len(time_series)):
            count = 0
            for step in range(0, len(self.time_series[i]), sample_step):
                self.data.append(self.time_series[i][step : step + self.W])
                self.indexed_labels.append(self.labels[i] + '_' + str(count))
                count += 1
        self.count = count

    def get_max(self):
        return [np.max(window) for window in self.data], [label + '_max' for label in self.indexed_labels]

    def get_min(self):
        return [np.min(window) for window in self.data], [label + '_min' for label in self.indexed_labels]

    def get_std(self):
        return [np.std(window) for window in self.data], [label + '_std' for label in self.indexed_labels]

    def get_ptp(self):
        return [np.ptp(window) for window in self.data], [label + '_ptp' for label in self.indexed_labels]

    def get_avg(self):
        return [np.average(window) for window in self.data], [label + '_avg' for label in self.indexed_labels]

    def get_median(self):
        return [np.median(window) for window in self.data], [label + '_median' for label in self.indexed_labels]

    def get_mean(self):
        return [np.mean(window) for window in self.data], [label + '_mean' for label in self.indexed_labels]

    def get_pos(self):
        return [len(window[window >= 0]) for window in self.data], [label + '_pos' for label in self.indexed_labels]

    def get_neg(self):
        return [len(window[window < 0]) for window in self.data], [label + '_neg' for label in self.indexed_labels]

    def get_iqr(self):
        return [np.percentile(window, 75) - np.percentile(window, 25) for window in self.data], [label + '_iqr' for label in self.indexed_labels]

    def get_over_avg(self):
        return [len(window[window > np.average(window)]) for window in self.data], [label + '_over_avg' for label in self.indexed_labels]

    def get_peaks(self):
        return [len(find_peaks(window)[0]) for window in self.data], [label + '_peaks' for label in self.indexed_labels]

    def get_skew(self):
        return [skew(window) for window in self.data], [label + '_skew' for label in self.indexed_labels]

    def get_kurtosis(self):
        return [kurtosis(window) for window in self.data], [label + '_kurtosis' for label in self.indexed_labels]

    def get_energy(self):
        return [np.sum(window ** 2) / self.W for window in self.data], [label + '_energy' for label in self.indexed_labels]

    def get_acc(self):
        return [np.average(np.sqrt(self.data[start + i] ** 2 +
                self.data[start + i + self.count] ** 2 +
                self.data[start + i + self.count * 2] ** 2))
                for start in range(0, len(self.data) , 3 * self.count)
                for i in range(self.count)], 'acc_'

    def get_aria(self):
        return [np.sum(np.abs(self.data[start + i]) +
                np.abs(self.data[start + i + self.count]) +
                np.abs(self.data[start + i + self.count * 2]))
                for start in range(0, len(self.data) , 3 * self.count)
                for i in range(self.count)], 'aria_'

    def get_all(self, no_3D = False):
        results_max, labels_max = self.get_max()
        results_min, labels_min = self.get_min()
        results_std, labels_std = self.get_std()
        results_ptp, labels_ptp = self.get_ptp()
        results_avg, labels_avg = self.get_avg()
        results_median, labels_median = self.get_median()
        results_mean, labels_mean = self.get_mean()
        results_pos, labels_pos = self.get_pos()
        results_neg, labels_neg = self.get_neg()
        results_iqr, labels_iqr = self.get_iqr()
        results_over_avg, labels_over_avg = self.get_over_avg()
        results_peaks, labels_peaks = self.get_peaks()
        results_skew, labels_skew = self.get_skew()
        results_kurtosis, labels_kurtosis = self.get_kurtosis()
        results_energy, labels_energy = self.get_energy()

        results = [*results_max, *results_min, *results_std, *results_ptp, *results_avg, *results_median, *results_mean,
                   *results_pos, *results_neg, *results_iqr, *results_over_avg, *results_peaks, *results_skew,
                   *results_kurtosis, *results_energy]
        final_labels = [*labels_max, *labels_min, *labels_std, *labels_ptp, *labels_avg, *labels_median, *labels_mean,
                        *labels_pos, *labels_neg, *labels_iqr, *labels_over_avg, *labels_peaks, *labels_skew,
                        *labels_kurtosis, *labels_energy]

        if no_3D == False:
            results_acc, label_acc = self.get_acc()
            results_aria, label_aria = self.get_aria()

            labels_acc = [label_acc + str(i) for i in range(len(results_acc))]
            labels_aria = [label_aria + str(i) for i in range(len(results_aria))]

            results = [*results, *results_acc, *results_aria]
            final_labels = [*final_labels, *labels_acc, *labels_aria]

        return {final_labels[i]:[results[i]] for i in range(len(results))}

In [ ]:
# RacketSports Test
time_series_x_acc = X_train_sports['dim_0'].to_numpy()[0].to_numpy()
time_series_y_acc = X_train_sports['dim_1'].to_numpy()[0].to_numpy()
time_series_z_acc = X_train_sports['dim_2'].to_numpy()[0].to_numpy()
time_series_x_giro = X_train_sports['dim_3'].to_numpy()[0].to_numpy()
time_series_y_giro = X_train_sports['dim_4'].to_numpy()[0].to_numpy()
time_series_z_giro = X_train_sports['dim_5'].to_numpy()[0].to_numpy()

time_series = [time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro]
time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

# fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])
fe = feature_extractor(time_series, ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5'])

results = fe.get_all()

features_sports_train = pd.DataFrame.from_dict(results)

for i in range(1, len(X_train_sports)):
    time_series_x_acc = X_train_sports['dim_0'].to_numpy()[i].to_numpy()
    time_series_y_acc = X_train_sports['dim_1'].to_numpy()[i].to_numpy()
    time_series_z_acc = X_train_sports['dim_2'].to_numpy()[i].to_numpy()
    time_series_x_giro = X_train_sports['dim_3'].to_numpy()[i].to_numpy()
    time_series_y_giro = X_train_sports['dim_4'].to_numpy()[i].to_numpy()
    time_series_z_giro = X_train_sports['dim_5'].to_numpy()[i].to_numpy()

    time_series = [time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro]
    time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])
    fe = feature_extractor(time_series, ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5'])

    results = fe.get_all()

    features_sports_train = pd.concat([features_sports_train, pd.DataFrame.from_dict(results)], ignore_index=True)

features_sports_train.to_csv('features_sports_train.csv')

# RacketSports Train
time_series_x_acc = X_test_sports['dim_0'].to_numpy()[0].to_numpy()
time_series_y_acc = X_test_sports['dim_1'].to_numpy()[0].to_numpy()
time_series_z_acc = X_test_sports['dim_2'].to_numpy()[0].to_numpy()
time_series_x_giro = X_test_sports['dim_3'].to_numpy()[0].to_numpy()
time_series_y_giro = X_test_sports['dim_4'].to_numpy()[0].to_numpy()
time_series_z_giro = X_test_sports['dim_5'].to_numpy()[0].to_numpy()

time_series = [time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro]
time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

fe = feature_extractor(time_series, ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5'])
# fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

results = fe.get_all()

features_sports_test = pd.DataFrame.from_dict(results)

for i in range(1, len(X_test_sports)):
    time_series_x_acc = X_test_sports['dim_0'].to_numpy()[i].to_numpy()
    time_series_y_acc = X_test_sports['dim_1'].to_numpy()[i].to_numpy()
    time_series_z_acc = X_test_sports['dim_2'].to_numpy()[i].to_numpy()
    time_series_x_giro = X_test_sports['dim_3'].to_numpy()[i].to_numpy()
    time_series_y_giro = X_test_sports['dim_4'].to_numpy()[i].to_numpy()
    time_series_z_giro = X_test_sports['dim_5'].to_numpy()[i].to_numpy()

    time_series = [time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro]
    time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    fe = feature_extractor(time_series, ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5'])
    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

    results = fe.get_all()

    features_sports_test = pd.concat([features_sports_test, pd.DataFrame.from_dict(results)], ignore_index=True)

features_sports_test.to_csv('features_sports_test.csv')


In [ ]:
# Mitbih train

time_series = X_train_mitbih.to_numpy()[0]
fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
results = fe.get_all(no_3D=True)
features_mitbih_train = pd.DataFrame.from_dict(results)

for i in range(1, len(X_train_mitbih)):
    print(i)
    time_series = X_train_mitbih.to_numpy()[i]

    # time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

    results = fe.get_all(no_3D=True)

    features_mitbih_train = pd.concat([features_mitbih_train, pd.DataFrame.from_dict(results)], ignore_index=True)

features_mitbih_train.to_csv('features_mitbih_train.csv')

# Mitbih test
time_series = X_test_mitbih.to_numpy()[0]
fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
results = fe.get_all(no_3D=True)
features_mitbih_test = pd.DataFrame.from_dict(results)

for i in range(1, len(X_test_mitbih)):
    time_series = X_test_mitbih.to_numpy()[i]

    # time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

    results = fe.get_all(no_3D=True)

    features_mitbih_test = pd.concat([features_mitbih_test, pd.DataFrame.from_dict(results)], ignore_index=True)

features_mitbih_test.to_csv('features_mitbih_test.csv')


In [ ]:
# ptbdb train
print(len(X_train_ptbdb))
for i in range(len(X_train_ptbdb)):
    print(i)
    time_series = X_train_ptbdb.to_numpy()[i]

    # time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

    results = fe.get_all(no_3D=True)

    if i == 0:
        features_ptbdb_train = pd.DataFrame.from_dict(results)
    else:
        features_ptbdb_train = pd.concat([features_ptbdb_train, pd.DataFrame.from_dict(results)], ignore_index=True)

features_ptbdb_train.to_csv('features_ptbdb_train.csv')

# Ptbdb test
for i in range(len(X_test_ptbdb)):
    time_series = X_test_ptbdb.to_numpy()[i]

    # time_series_fft = np.fft.fft([time_series_x_acc,  time_series_y_acc, time_series_z_acc, time_series_x_giro,  time_series_y_giro, time_series_z_giro])

    fe = feature_extractor([time_series], ['series'], W = 100, sample_step=12)
    # fe = feature_extractor([*time_series, *time_series_fft], ['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_0_fft', 'dim_1_fft', 'dim_2_fft', 'dim_3_fft', 'dim_4_fft', 'dim_5_fft'])

    results = fe.get_all(no_3D=True)

    if i == 0:
        features_ptbdb_test = pd.DataFrame.from_dict(results)
    else:
        features_ptbdb_test = pd.concat([features_mitbih_test, pd.DataFrame.from_dict(results)], ignore_index=True)

features_ptbdb_test.to_csv('features_ptbdb_test.csv')


In [ ]:
functins = ['self.get_max()', 'self.get_min()', 'self.get_std()', 'self.get_ptp()', 'self.get_avg()', 'self.get_median()', 'self.get_mean()', 'self.get_pos()', 'self.get_neg()', 'self.get_iqr()', 'self.get_over_avg()', 'self.get_peaks()', 'self.get_skew()', 'self.get_kurtosis()', 'self.get_energy()', 'self.get_acc()', 'self.get_aria()']
names = ['_max', '_min', '_std', '_ptp', '_avg', '_median', '_mean', '_pos', '_neg', '_iqr', '_over_avg', '_peaks', '_skew', '_kurtosis', '_energy', '_acc', '_aria']

for i in range(len(functins)):
    # print('results' + names[i] + ', labels' + names[i] + ' = ' + functins[i])
    # print('*results' + names[i] + ',', end=" ")
    print('*labels' + names[i] + ',', end=" ")


In [ ]:
rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                          min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None,
                           verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)


le = LabelEncoder().fit(y_train_sports)
y_sports_train_enc = le.transform(y_train_sports)

le = LabelEncoder().fit(y_test_sports)
y_sports_test_enc = le.transform(y_test_sports)

rf.fit(features_sports_train, y_sports_train_enc)
y_pred_sports = rf.predict(features_sports_test)
y_pred_proba_sports = rf.predict_proba(features_sports_test)
print(y_sports_test_enc)
print(y_pred_sports)
print(y_pred_proba_sports)

In [ ]:
svm = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False,
          tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr',
            break_ties=False, random_state=None)

svm.fit(features_sports_train, y_sports_train_enc)
y_pred_svm_sports = svm.predict(features_sports_test)
print(y_sports_test_enc)
print(y_pred_svm_sports)

In [39]:
shutil.rmtree('ECG', ignore_errors=True)
shutil.rmtree('RacketSports', ignore_errors=True)